In [2]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [4]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))]) # Ovdje definise outputs, stavlja da je output zbir 2 inputa
print(in_seq1)
print(in_seq2)
print(out_seq)

[10 20 30 40 50 60 70 80 90]
[15 25 35 45 55 65 75 85 95]
[ 25  45  65  85 105 125 145 165 185]


In [5]:
# convert to [rows, columns] structure
# U sustini radi reshape da bude jedna kolona, a brojevi poslozeni u rows
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
print(in_seq1)
print(in_seq2)
print(out_seq)

[[10]
 [20]
 [30]
 [40]
 [50]
 [60]
 [70]
 [80]
 [90]]
[[15]
 [25]
 [35]
 [45]
 [55]
 [65]
 [75]
 [85]
 [95]]
[[ 25]
 [ 45]
 [ 65]
 [ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]


In [6]:
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
#As the comment implies, stacks the columns left to right
# in_seq1 in_seq2 out_seq
# [[ 10  15  25]
#  [ 20  25  45]
#  [ 30  35  65]
#  [ 40  45  85]
#  [ 50  55 105]
#  [ 60  65 125]
#  [ 70  75 145]
#  [ 80  85 165]
#  [ 90  95 185]]

print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [12]:
# choose a number of time steps
# n_steps_in je input koliko ih ima, n_step_out je forecast period
n_steps_in, n_steps_out = 3, 3 # U sustini ovdje se stima forecast period, gdje n_steps_in predstavlja na osnovu koliko dana ce forecast
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2] #Broj kolona
print(X) #X je array sequenca, tj inout stepova. 
print('This is X[0]')
print(X[0])
print('This is y') #Y je set odgovora, svaki row je zapravo jedan odgovor.
print(y)
print(n_features)

[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]]
This is X[0]
[[10 15]
 [20 25]
 [30 35]]
This is y
[[ 65  85 105]
 [ 85 105 125]
 [105 125 145]
 [125 145 165]
 [145 165 185]]
2


In [13]:
# Model kod iz drugog fajla
# Create the model
# inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
# x = keras.layers.LSTM(150, return_sequences= True)(inputs)
# x = keras.layers.Dropout(0.3)(x)
# x = keras.layers.LSTM(150, return_sequences=True)(x)
# x = keras.layers.Dropout(0.3)(x)
# x = keras.layers.LSTM(150)(x)
# outputs = keras.layers.Dense(1, activation='linear')(x)

# model = keras.Model(inputs=inputs, outputs=outputs)
# model.compile(optimizer='adam', loss="mse")
# model.summary()

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 3, 100)              │          41,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             303 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 121,903 (476.18 KB)

 Trainable params: 121,903 (476.18 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
# Nemoj zaboraviti da jedan row je jedan step, a kolona je jedan feature
x_input = array([[70, 75], 
                 [80, 85], 
                 [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[188.55278 214.90152 237.58797]]
